In [1]:
import torch
from torch.utils.data import DataLoader

from dense_retrieval import DenseRetrieval

In [2]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from typing import List, Tuple, NoReturn, Any, Optional, Union

import torch
import torch.nn.functional as F
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

from transformers import AutoTokenizer, BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup
from datasets import Dataset, load_from_disk, concatenate_datasets

from retrieval import SparseRetrieval, timer

In [3]:
import os
import json 

data_path  = "../data/"
dataset_path = "../data/train_dataset"
context_path = "wikipedia_documents.json"
model_checkpoint = "klue/bert-base"

org_dataset = load_from_disk(dataset_path)
full_ds = concatenate_datasets([
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ])

with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)
contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))

Loading cached processed dataset at ../data/train_dataset/train/cache-fbc57aa6e699fb0c.arrow
Loading cached processed dataset at ../data/train_dataset/validation/cache-d2fba0c42123b1d6.arrow


In [4]:
len(contexts)

56737

In [5]:
[s for s in contexts if org_dataset["validation"][0]['context'][0:5] in s]

['순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\n\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다.  \n\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
 '순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,use_fast=False,)

args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=40,
    weight_decay=0.01,
)

In [17]:
dense_retriever = DenseRetrieval(tokenize_fn=tokenizer.tokenize, data_path = data_path, 
                                context_path = context_path, dataset_path=dataset_path, 
                                tokenizer=tokenizer, train_data=org_dataset["validation"])
## 학습과정 ##
# train_dataset = dense_retriever.make_train_data(tokenizer)
# dense_retriever.init_model(model_checkpoint)
# dense_retriever.train(args, train_dataset)

## 추론과정 ##
dense_retriever.load_model(model_checkpoint, "outputs/p_encoder_15.pt", "outputs/q_encoder_15.pt")
dense_retriever.get_dense_embedding()
#df = dense_retriever.retrieve(full_ds[0]['question'], topk=3)

Lengths of unique contexts : 56737
Embedding pickle load.
cuda:0


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bi

load_model finished...


100%|██████████| 60/60 [00:04<00:00, 14.37it/s]

get_dense_embedding finished...


In [18]:
org_dataset["validation"][0]

{'title': '전효숙',
 'context': '순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
 'question': '처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 'id': 'mrc-0-003264',
 'answers': {'answer_start': [284], 'text': ['한보철강']},
 'document_id': 9027,
 '__index_level_0__': 2146}

In [19]:
result_retriever = dense_retriever.retrieve(org_dataset["validation"], topk=5)

Sparse retrieval:   6%|▌         | 14/240 [00:00<00:03, 65.37it/s]

[query exhaustive search] done in 3.197 s


Sparse retrieval: 100%|██████████| 240/240 [00:03<00:00, 67.09it/s]


In [20]:
result_retriever

,question,id,context_id,context,original_context,answers
0,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"[159, 122, 19, 209, 61]","텍사스 혁명\n1830년에서 1832년, 멕시코 의회는 코아우일라 테하스 주에서 영...","순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","{'answer_start': [284], 'text': ['한보철강']}"
1,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"[159, 122, 209, 153, 135]","텍사스 혁명\n1830년에서 1832년, 멕시코 의회는 코아우일라 테하스 주에서 영...",요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,"{'answer_start': [146], 'text': ['1871년']}"
2,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"[159, 199, 209, 71, 122]","텍사스 혁명\n1830년에서 1832년, 멕시코 의회는 코아우일라 테하스 주에서 영...","촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","{'answer_start': [517], 'text': ['나뭇잎']}"
3,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"[159, 209, 122, 153, 61]","텍사스 혁명\n1830년에서 1832년, 멕시코 의회는 코아우일라 테하스 주에서 영...",프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,"{'answer_start': [1109], 'text': ['금대야']}"
4,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"[122, 25, 199, 159, 153]",초기 연구자들은 귀인 편향을 인지적으로 주도된 정보처리 오류의 산물로서 설명했다. ...,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,"{'answer_start': [386], 'text': ['수평적 관계']}"
...,...,...,...,...,...,...
235,전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,mrc-0-000484,"[122, 159, 163, 209, 25]",초기 연구자들은 귀인 편향을 인지적으로 주도된 정보처리 오류의 산물로서 설명했다. ...,"연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...","{'answer_start': [1084], 'text': ['제 양왕']}"
236,공놀이 경기장 중 일부는 어디에 위치하고 있나?,mrc-0-002095,"[9, 206, 125, 114, 19]",여러 유적이 로마 제국의 영역상 헤자즈 일대에 미쳤음을 증명하고 있다. 이집트와 오...,현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,"{'answer_start': [343], 'text': [''일곱 개의 신전 광장..."
237,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,mrc-0-003083,"[159, 122, 209, 61, 19]","텍사스 혁명\n1830년에서 1832년, 멕시코 의회는 코아우일라 테하스 주에서 영...",1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,"{'answer_start': [247], 'text': ['미나미 지로']}"
238,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,mrc-0-002978,"[159, 209, 141, 34, 135]","텍사스 혁명\n1830년에서 1832년, 멕시코 의회는 코아우일라 테하스 주에서 영...",빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,"{'answer_start': [563], 'text': ['200,000명']}"


import pickle

with open("./data/dense_embedding.bin", "wb") as f:
    pickle.dump(dense_retriever.p_embedding, f)

In [21]:
topK_list = [10,20,50]
result = dense_retriever.topk_experiment(topK_list, org_dataset["validation"])
print(result)

100%|██████████| 60/60 [00:03<00:00, 19.81it/s]


[query exhaustive search] done in 3.047 s


100%|██████████| 60/60 [00:02<00:00, 20.25it/s]


[query exhaustive search] done in 2.978 s


100%|██████████| 60/60 [00:03<00:00, 19.43it/s]



[query exhaustive search] done in 3.105 s


100%|██████████| 3/3 [01:27<00:00, 29.09s/it]

{10: 0.12083333333333333, 20: 0.2125, 50: 0.45}
